In [ ]:
cd ..

In [ ]:
import numpy as np

import util.io

In [ ]:
image_data = util.io.load_json('./exp-visgeno-rel/visgeno-dataset/image_data.json')
objects = util.io.load_json('./exp-visgeno-rel/visgeno-dataset/objects.json')
relationships = util.io.load_json('./exp-visgeno-rel/visgeno-dataset/relationships.json')

In [ ]:
# compute average object number per image
count = 0
for obj in objects:
    count += len(obj['objects'])
    
print('average objects per image:', count / len(objects))

In [ ]:
# compute average relationship number per image
count = 0
for rel in relationships:
    count += len(rel['relationships'])
    
print('average relationships per image:', count / len(relationships))

In [ ]:
# collect a list of objects per image (turn into bbox format)
# (obj_idx_map, bboxes)
imdb = [None for _ in range(len(objects))]

In [ ]:
def _convert_bbox(obj):
    x, y, w, h = obj['x'], obj['y'], obj['w'], obj['h']
    bbox = [x, y, x+w-1, y+h-1]
    return bbox

for n_im in range(len(objects)):
    img_info = image_data[n_im]
    assert(img_info['image_id'] == objects[n_im]['image_id'])
    
    objs = objects[n_im]['objects']
    # a dict mapping object_id to index in the list
    obj_idx_map = {objs[n_obj]['object_id']: n_obj for n_obj in range(len(objs))}
    # bounding box of each object
    obj_bboxes = [_convert_bbox(obj) for obj in objs]
        
    im_name = img_info['url'].replace('https://cs.stanford.edu/people/rak248/', '')
    im_path = './exp-visgeno-rel/visgeno-dataset/' + im_name

    imdb[n_im] = dict(image_id=img_info['image_id'], 
                      obj_idx_map=obj_idx_map,
                      bboxes=obj_bboxes,
                      im_path=im_path)

In [ ]:
# match the relationships to objects
for n_im in range(len(relationships)):
    img_info = image_data[n_im]
    assert(img_info['image_id'] == relationships[n_im]['image_id'])
    
    rels = relationships[n_im]['relationships']
    obj_idx_map = imdb[n_im]['obj_idx_map']
    
    mapped_rels = []
    for rel in rels:
        subj_name = rel['subject']['name']
        obj_name = rel['object']['name']
        predcate_name = rel['predicate']
        subj_idx = obj_idx_map[rel['subject']['object_id']]
        obj_idx = obj_idx_map[rel['object']['object_id']]
        mapped_rels.append((subj_idx, obj_idx, subj_name, predcate_name, obj_name))
        
    imdb[n_im]['mapped_rels'] = mapped_rels

In [ ]:
partition_file = './exp-visgeno-rel/data/densecap_splits.json'
splits = util.io.load_json(partition_file)

In [ ]:
rel_data_trn = [rel for rel in rel_data if rel['image_id'] in splits['train']]
rel_data_val = [rel for rel in rel_data if rel['image_id'] in splits['val']]
rel_data_tst = [rel for rel in rel_data if rel['image_id'] in splits['test']]

In [ ]:
util.io.save_numpy_obj(rel_data_trn, './exp-visgeno-rel/data/imdb/imdb_trn.npy')
util.io.save_numpy_obj(rel_data_val, './exp-visgeno-rel/data/imdb/imdb_val.npy')
util.io.save_numpy_obj(rel_data_tst, './exp-visgeno-rel/data/imdb/imdb_tst.npy')